# Error bars

Given the formula that links the differential cross section to the event rate measured by our experiment:

$$
\frac{dR}{d \Omega} = \epsilon_{spectrometer}\left(\theta \right) \cdot \frac{n_{gate}}{T} \cdot n_{c} \cdot \wp(\theta; \lambda', \lambda '') \cdot \frac{d \sigma}{d \Omega}
$$

it is crucial to correctly evaluate the uncertainities of these measurements. 

## Statistical and systematical uncertainties + work flow
First thing first it is fundamental to distinguish between the various sources of errors one can come across during a measurement in a nuclear physics' experiment.
In particular we need to separate the contributions given by $\textit{statistical}$ and $\textit{systematical}$ uncertainties:

- Statistical uncertainities could come from the counting of photons or the errors associated to the fit parameters, a square sum will be used to take account of them.
- Systematical uncertainties arise from various effects, such as the shift of the Compton peak, the errors associated with the geometrical measurements of the apparatus and so on, we'll take account of them with a square sum as well.

By looking at the expression above one could come up with an expression for the uncertainty that looks like:

$$
\left(\frac{\Delta R}{R} \right)^2 = \left(\frac{\Delta \epsilon_{spectrometer}}{\epsilon_{spectrometer}} \right)^2 + \left(\frac{\Delta n_{gate}}{n_{gate}} \right)^2 + \left(\frac{\Delta n_c}{n_c} \right)^2 + \left(\frac{\Delta \wp}{\wp} \right)^2 + \left(\frac{\Delta T}{T} \right)^2
$$

assuming every quantity considered in this expression is independent from one another and has a elatively small error associated to it.

To make sure randomness and bias are not mixed together these two kinds of uncertainties will be combined as:

$$
\sigma_{tot} = \sigma_{statistic} + \sigma_{systematic}
$$

so that one can split the two contributes in:

$$ \left(\frac{\Delta \sigma_{stat}}{\sigma} \right)^2 = \sum_{X \, \in \, (\epsilon_{spectr.}, n_{gate}, T, n_c, \wp)}  \left(\frac{\Delta X_{stat}}{X} \right)^2 $$
$$ \left(\frac{\Delta \sigma_{sist}}{\sigma} \right)^2 = \sum_{X \, \in \, (\epsilon_{spectr.}, n_{gate}, T, n_c, \wp)}  \left(\frac{\Delta X_{sist}}{X} \right)^2 $$


## Scattering centers' density uncertainty

Given:

$$
n_c = \rho \frac{N_a}{MM}
$$

where: 
- $\rho$ is the density of the scattering target
- $N_a$ is the Avogadro number
- $\text{MM}$ is the molar mass of the scattering target

One can clearly see that all these quantities are known with extreme accuracy, we won't take account of their uncertainties.

## $\epsilon_{spectrometer}$ uncertainty

In this case the value of $\epsilon_{spectrometer}$ is obtained by an interpolation, the error associated to it will then be consider as a statistical uncertainty. Given the formula used to fit the efficiency curve: 

$$
\epsilon_{spectrometer} = A \cdot E^{-B} \cdot \exp(-C \cdot E) + D
$$

where $A,B,C,D$ are all model parameters and $E = \sigma_{Compton} \times 10^{-3}$, which is considered without uncertainty since $\sigma_{Compton}$ comes from theory. Given that, one can compute the error associated to this quantity as:

$$
\Delta \epsilon = \sqrt{\left( \frac{\partial \epsilon}{\partial A} \cdot \Delta A \right)^2 + \left( \frac{\partial \epsilon}{\partial B} \cdot \Delta B \right)^2 + \left( \frac{\partial \epsilon}{\partial C} \cdot \Delta C \right)^2 + \left( \frac{\partial \epsilon}{\partial D} \cdot \Delta D \right)^2}
$$

(QUESTO ASSUMENDO CHE NON CI SIA CORRELAZIONE TRA I PARAMETRI DEL FIT)


## Gate's photons uncertainty

Given: 

$$
n_{gate} = 2 \cdot S \left(t \right) \cdot \text{BR} \cdot \frac{\Delta\Omega}{4\pi} \cdot \epsilon_{gate}(511)
$$

where: 
- $S(t)$ is the activity of the source in Bq
- $\text{BR}$ is the branching ratio of the 511 keV photon
- $\frac{\Delta\Omega}{4\pi}$ is the solid angle
- $\epsilon_{gate}(511)$ is the efficiency of the gate detector for the 511 keV photon

In this case one can assume that the branching ratio (BR) is well known, the uncertainties linked to $S \left(t \right)$ and $\epsilon_{gate}$ are statistical errors and the one associated to $\Delta \Omega$ is systematical.

### $\Delta S(t)$ computation

This uncertainty has been computed from (CHIEDERE A RICKY).
The associated error is then:

$$
\sigma_{S \left( t \right)} = 10491 \, Bq.
$$

### $\Delta \epsilon_{gate}$ computation

For this computation please refer to the previous section. 


### $\Delta (\Delta \Omega)$ computation

For the uncertainty linked to the measurement of the solid angle covered by the spectrometer one can start from the formula:

$$
\Delta \Omega = 2 \pi \cdot \left(1 - \cos (\beta) \right)
$$

where $\beta$ is computed as: 

$$
\beta = \arctan \left( \frac{r_{gate}}{d_{source-gate}} \right).
$$

Both of these two measurements are known with their uncertainty, $r_{gate} = r =  (0.62 \pm 0.05) \, in = (1,575 \pm 0,127) \, cm$ and ${d_{source-gate}} = d =(16.0 \pm 0.1) \, cm$, knowing that one can propagate the errors using the following expression:

$$
\left(\Delta (\Delta \Omega) \right)^2 = \left( \frac{\partial (\Delta \Omega)}{\partial r} \Delta r\right)^2 + \left( \frac{\partial (\Delta \Omega)}{\partial d} \Delta d\right)^2
$$

(IN REALTA' ABBIAMO CHE L'INCERTEZZA SU R DOVREBBE ESSERE STATISTICA PERCHE' FORNITA DAL MANUALE ORTEC QUINDI PRESUMIBILMENTE RPOVENIENTE DA MISURE RIPETUE, MENTRE QUELLA LEGATA ALLA DISTANZA DIREI CHE E' SISTEMATICA PERCHE' VIENE DA UNA SOLA NOSTRA MISURA FATTA A CASO E POI LA DISTANZA GATE SOURCE E' SOGGETTA A FLUTTUAZIONI DOVUTE ALLO SPOSTAMENTO DELL'APPARATO E ROBE VARIE)
and that:

$$
\frac{\partial (\Delta \Omega)}{\partial r} = 2 \pi \sin(\beta) \frac{\partial \beta}{\partial r} = 2 \pi \frac{r \cdot d}{(r^2 + d^2)^{3/2}}
$$

$$
\frac{\partial (\Delta \Omega)}{\partial d} = 2 \pi \sin(\beta) \frac{\partial \beta}{\partial d} = - 2 \pi \frac{r^2}{(r^2 + d^2)^{3/2}}
$$

one can find the uncertainty asssociated to the solid angle using the formula below:

$$
\Delta \left(\Delta \Omega \right) = \sqrt{\left(2 \pi \frac{r \cdot d}{(r^2 + d^2)^{3/2}} \cdot \Delta r \right)^2 + \left(2 \pi \frac{r^2}{(r^2 + d^2)^{3/2}} \cdot \Delta d \right)^2}
$$

obtaining:

$$
\Delta(\Delta \Omega) = 3.65 \times 10^{-3} sr
$$





For each of them we get the following values:
- $\sigma_{S \left( t \right)} = 10 491 Bq$ 
- $\sigma_{\epsilon_{gate}} = $
- $\sigma_{\Delta \Omega} = 3.65 \times 10^{-3} sr$